In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import glob
import os
import sys
from PIL import Image

In [23]:
masks = glob.glob("masks/*.png")
orgs = glob.glob("images/*.png")

In [86]:
imgs_list = []
masks_list = []

for image, mask in zip(orgs, masks):
    i_np = np.array(Image.open(image).resize((384,384)))
    imgs_list.append(i_np)
    m_np = np.array(Image.open(mask).resize((384,384)))
    if m_np.shape != (384, 384, 3):
#         print(i_np.shape, m_np.shape)
        m_np = np.broadcast_to(m_np[..., np.newaxis], (384,384, 3))
#     m_np = m_np[:, :, np.newaxis]
    masks_list.append(m_np)
    
    
imgs_np = np.array(imgs_list)
masks_np = np.array(masks_list)

In [87]:
x = np.asarray(imgs_list, dtype=np.float32)

In [88]:
y = np.asarray(masks_list, dtype=np.float32)

In [89]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.9, random_state=0)


print("x_train: ", x_train.shape)
print("y_train: ", y_train.shape)
print("x_val: ", x_val.shape)
print("y_val: ", y_val.shape)

x_train:  (121, 384, 384, 3)
y_train:  (121, 384, 384, 3)
x_val:  (1098, 384, 384, 3)
y_val:  (1098, 384, 384, 3)


In [90]:
for i in y_val:
    if i.shape != (384, 384, 3):
        print(' ffffffff')

In [91]:
x_val[0].shape

(384, 384, 3)

In [92]:

from keras_unet.models import custom_unet

input_shape = x_train[0].shape

model = custom_unet(
    input_shape,
    filters=32,
    use_batch_norm=True,
    dropout=0.3,
    dropout_change_per_layer=0.0,
    num_layers=4
)

In [93]:

from keras.callbacks import ModelCheckpoint


model_filename = 'segm_model_v3.h5'
callback_checkpoint = ModelCheckpoint(
    model_filename, 
    verbose=1, 
    monitor='val_loss', 
    save_best_only=True,
)

In [94]:
from keras.optimizers import Adam, SGD
from keras_unet.metrics import iou, iou_thresholded
from keras_unet.losses import jaccard_distance

model.compile(
#     optimizer=Adam(), 
    optimizer=SGD(lr=0.01, momentum=0.99),
#     loss='binary_crossentropy',
    loss=jaccard_distance,
    metrics=[iou, iou_thresholded]
)

In [95]:
x_val.shape

(1098, 384, 384, 3)

In [96]:

history = model.fit(
    x_train, y_train,
    steps_per_epoch=200,
    epochs=50,
    
    validation_data=(x_val, y_val),
    callbacks=[callback_checkpoint]
)

Epoch 1/50


ValueError: in user code:

    c:\python38\lib\site-packages\tensorflow\python\keras\engine\training.py:805 train_function  *
        return step_function(self, iterator)
    c:\python38\lib\site-packages\keras_unet\metrics.py:13 iou  *
        intersection = K.sum(y_true_f * y_pred_f)
    c:\python38\lib\site-packages\tensorflow\python\ops\math_ops.py:1180 binary_op_wrapper
        raise e
    c:\python38\lib\site-packages\tensorflow\python\ops\math_ops.py:1164 binary_op_wrapper
        return func(x, y, name=name)
    c:\python38\lib\site-packages\tensorflow\python\ops\math_ops.py:1496 _mul_dispatch
        return multiply(x, y, name=name)
    c:\python38\lib\site-packages\tensorflow\python\util\dispatch.py:201 wrapper
        return target(*args, **kwargs)
    c:\python38\lib\site-packages\tensorflow\python\ops\math_ops.py:518 multiply
        return gen_math_ops.mul(x, y, name)
    c:\python38\lib\site-packages\tensorflow\python\ops\gen_math_ops.py:6076 mul
        _, _, _op, _outp

In [ ]:
from keras_unet.utils import plot_segm_history

plot_segm_history(history)